In [ ]:
import sympy as sp
nq_arm = 7

In [ ]:
alg_inputs = []

# PROGRAMMING: Do this with Drake instead?
# PROGRAMMING: Can I get some of these lists from sympy?
# Physical and geometric quantities
m_L = sp.symbols(r"m_L")
alg_inputs.append(m_L)
w_L = sp.symbols(r"w_L")
alg_inputs.append(w_L)
I_L = sp.symbols(r"I_L")
alg_inputs.append(I_L)
h_L = sp.symbols(r"h_L")
alg_inputs.append(h_L)
r = sp.symbols(r"r")
alg_inputs.append(r)

# Friction coefficients
mu = sp.symbols(r"\mu")
alg_inputs.append(mu)
mu_S = sp.symbols(r"\mu_{S}")
alg_inputs.append(mu_S)
hats_T = sp.symbols(r"\hat{s}_T")
alg_inputs.append(hats_T)

# System gains
b_J = sp.symbols(r"b_J")
alg_inputs.append(b_J)
k_J = sp.symbols(r"k_J")
alg_inputs.append(k_J)

# Positions
p_CN = sp.symbols(r"p_{CN}")
alg_inputs.append(p_CN)
p_CT = sp.symbols(r"p_{CT}")
alg_inputs.append(p_CT)
p_MN = sp.symbols(r"p_{MN}")
alg_inputs.append(p_MN)
p_LN = sp.symbols(r"p_{LN}")
alg_inputs.append(p_LN)
p_LT = sp.symbols(r"p_{LT}")
alg_inputs.append(p_LT)
theta_L = sp.symbols(r"\theta_L")
alg_inputs.append(theta_L)
d_T = sp.symbols(r"d_T")
alg_inputs.append(d_T)
d_N = sp.symbols(r"d_N")
alg_inputs.append(d_N)

# Derived terms
cos_theta_L = sp.symbols(r"\cos\theta_L")
sin_theta_L = sp.symbols(r"\sin\theta_L")

# Velocities
v_MN = sp.symbols(r"v_{MN}")
alg_inputs.append(v_MN)
v_MT = sp.symbols(r"v_{MT}")
alg_inputs.append(v_MT)
v_LN = sp.symbols(r"v_{LN}")
alg_inputs.append(v_LN)
v_LT = sp.symbols(r"v_{LT}")
alg_inputs.append(v_LT)
d_theta_L = sp.symbols(r"\dot\theta_L")
alg_inputs.append(d_theta_L)
d_theta_M = sp.symbols(r"\dot\theta_M")
alg_inputs.append(d_theta_M)
d_d_T = sp.symbols(r"\dot{d}_T")
alg_inputs.append(d_d_T)
d_d_N = sp.symbols(r"\dot{d}_N")
alg_inputs.append(d_d_N)

# Input forces
F_GT = sp.symbols(r"F_{GT}")
alg_inputs.append(F_GT)
F_GN = sp.symbols(r"F_{GN}")
alg_inputs.append(F_GN)
F_OT, F_ON, tau_O = sp.symbols(r"F_{OT}, F_{ON} \tau_O")
alg_inputs.append(F_OT)
alg_inputs.append(F_ON)
alg_inputs.append(tau_O)

# Control inputs
a_LNd = sp.symbols(r"a_{LNd}")
alg_inputs.append(a_LNd)

In [ ]:
M = sp.MatrixSymbol('M', nq_arm, nq_arm).as_explicit()
alg_inputs += list(M)
J = sp.MatrixSymbol('J', 2, nq_arm).as_explicit()
alg_inputs += list(J)

Cv = sp.MatrixSymbol('Cv', nq_arm, 1).as_explicit()
alg_inputs += list(Cv)
q = sp.MatrixSymbol('q', nq_arm, 1).as_explicit()
alg_inputs += list(q)
dq = sp.MatrixSymbol(r'\dot{q}', nq_arm, 1).as_explicit()
alg_inputs += list(dq)
tau_contact = sp.MatrixSymbol(r'{\tau_{contact}}', nq_arm, 1).as_explicit()
alg_inputs += list(tau_contact)
tau_ctrl = sp.MatrixSymbol(r'{\tau_{ctrl}}', nq_arm, 1).as_explicit()
alg_inputs += list(tau_ctrl)
tau_g = sp.MatrixSymbol(r'{\tau_{g}}', nq_arm, 1).as_explicit()
alg_inputs += list(tau_g)

Jdot_qdot = sp.MatrixSymbol('{\dot{J}\dot{q}}', 2, 1).as_explicit()
alg_inputs += list(Jdot_qdot)

In [ ]:
outputs = [
    F_CY, F_CZ, a_MT, a_MY, a_MZ, a_MN, a_LT, dd_theta_L, F_NM, F_FL, F_FM, tau_M, a_LN, dd_d_N, dd_d_T, F_NL, F_CN, F_CT,
] = list(sp.symbols(
    r"F_{CY}, F_{CZ}, a_{MY}, a_{MZ}, a_{MT}, a_{MN}, a_{LT}, \ddot\theta_L, F_{NM}, F_{FL}, F_{FM}, a_{LN}, \ddot\theta_M, \ddot{d}_N, \ddot{d}_T, F_{NL}, F_{CN}, F_{CT}"
))
ddq = sp.MatrixSymbol('\ddot{q}', nq_arm, 1).as_explicit()
outputs = list(ddq) + outputs

In [ ]:
outputs

In [ ]:
# For convenience
F_CYZ = sp.Matrix([F_CY, F_CZ])

In [ ]:
F_CYZ

In [ ]:
t = sp.symbols("t")
theta_L_func = sp.Function(r'\theta_L')(t)
N_hat = sp.Function(r'\hat N')(theta_L_func)
T_hat = sp.Function(r'\hat T')(theta_L_func)

d_T_func = sp.Function(r"d_T")(t)
d_N_func = sp.Function(r"d_N")(t)
d_g = d_T_func*T_hat + d_N_func*N_hat

d_vel_g = sp.diff(d_g, t)

d_vel_g = d_vel_g.subs(sp.diff(N_hat, t), -
                       sp.diff(theta_L_func, t)*T_hat)
d_vel_g = d_vel_g.subs(
    sp.diff(T_hat, t), sp.diff(theta_L_func, t)*N_hat)

d_acc_g = sp.diff(d_vel_g, t)
d_acc_g = d_acc_g.subs(sp.diff(N_hat, t), -
                       sp.diff(theta_L_func, t)*T_hat)
d_acc_g = d_acc_g.subs(
    sp.diff(T_hat, t), sp.diff(theta_L_func, t)*N_hat)

d_acc_cos_g = d_acc_g
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(theta_L_func, t, t), dd_theta_L)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(d_T_func, t, t), dd_d_T)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(d_N_func, t, t), dd_d_N)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(theta_L_func, t), d_theta_L)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(d_T_func, t), d_d_T)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(d_N_func, t), d_d_N)
d_acc_cos_g = d_acc_cos_g.subs(d_T_func, d_T)
d_acc_cos_g = d_acc_cos_g.subs(d_N_func, d_N)

dd_d_g_T = d_acc_cos_g.subs(N_hat, 0).subs(T_hat, 1)

dd_d_g_N = d_acc_cos_g.subs(T_hat, 0).subs(N_hat, 1)

p_M_func = sp.Function(r"p_M")(t)
p_L_func = sp.Function(r"p_L")(t)
v_M = sp.symbols(r"v_M")
v_L = sp.symbols(r"v_L")
d_s = (p_M_func + r*N_hat) - (p_L_func + (w_L/2)*T_hat - (h_L/2)*N_hat)

d_vel_s = sp.diff(d_s, t)
d_vel_s = d_vel_s.subs(sp.diff(N_hat, t), -
                       sp.diff(theta_L_func, t)*T_hat)
d_vel_s = d_vel_s.subs(
    sp.diff(T_hat, t), sp.diff(theta_L_func, t)*N_hat)

d_acc_s = sp.diff(d_vel_s, t)
d_acc_s = d_acc_s.subs(sp.diff(N_hat, t), -
                       sp.diff(theta_L_func, t)*T_hat)
d_acc_s = d_acc_s.subs(
    sp.diff(T_hat, t), sp.diff(theta_L_func, t)*N_hat)

d_acc_cos_s = d_acc_s
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(theta_L_func, t, t), dd_theta_L)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(d_T_func, t, t), dd_d_T)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(d_N_func, t, t), dd_d_N)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(theta_L_func, t), d_theta_L)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(d_T_func, t), d_d_T)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(d_N_func, t), d_d_N)
d_acc_cos_s = d_acc_cos_s.subs(d_T_func, d_T)
d_acc_cos_s = d_acc_cos_s.subs(d_N_func, d_N)

dd_d_s_T = d_acc_cos_s.subs(N_hat, 0).subs(T_hat, 1)
dd_d_s_T = dd_d_s_T.subs(sp.diff(p_M_func, t, t), a_MT)
dd_d_s_T = dd_d_s_T.subs(sp.diff(p_L_func, t, t), a_LT)
dd_d_s_T

dd_d_s_N = d_acc_cos_s.subs(T_hat, 0).subs(N_hat, 1)
dd_d_s_N = dd_d_s_N.subs(sp.diff(p_M_func, t, t), a_MN)
dd_d_s_N = dd_d_s_N.subs(sp.diff(p_L_func, t, t), a_LN)
dd_d_s_N

In [ ]:
nat_eqs = [
    # Link tangential force balance
    [m_L*a_LT, F_FL+F_GT+F_OT],
    # Link normal force balance
    [m_L*a_LN, F_NL + F_GN + F_ON, ],
    # Link moment balance
    [I_L*dd_theta_L, (-w_L/2)*F_ON - (p_CN-p_LN) * \
     F_FL + (p_CT-p_LT)*F_NL + tau_O, ],
    # 3rd law normal forces
    [F_NL, -F_NM],
    # Friction relationship L
    [F_FL, mu*mu_S*F_NL*hats_T],
    # Friction relationship M
    [F_FM, -F_FL],
    # d_T derivative is derivative
    [dd_d_s_T, dd_d_g_T],
    # d_N derivative is derivative
    [dd_d_s_N, dd_d_g_N],
    # No penetration
    [dd_d_N, 0],
    # Relate manipulator and end effector with joint velocities in Y direction
    [a_MY, (Jdot_qdot + J*ddq)[0]],
    # Relate manipulator and end effector with joint velocities in Z direction
    [a_MZ, (Jdot_qdot + J*ddq)[1]],
    # Projection equations
    [a_MT, cos_theta_L*a_MY + sin_theta_L*a_MZ],
    [a_MN, -sin_theta_L*a_MY + cos_theta_L*a_MZ],
    [F_CT, cos_theta_L*F_CY + sin_theta_L*F_CZ],
    [F_CN, -sin_theta_L*F_CY + cos_theta_L*F_CZ],
]
# Manipulator equations
for lhs, rhs in zip(M*ddq + Cv, tau_g +tau_contact + tau_ctrl):
    nat_eqs.append([lhs, rhs])
    break
# How controller force is converted to torques
for lhs, rhs in zip(tau_ctrl, J.T*F_CYZ):
    nat_eqs.append([lhs, rhs])
env_eqs = nat_eqs

In [ ]:
A = []
b = []
for lhs, rhs in env_eqs:
    A_row = []
    b_term = rhs - lhs
    for output_term in outputs:
        try:
            coeff_L = lhs.coeff(output_term)
        except AttributeError:
            coeff_L = 0
        try:
            coeff_R = rhs.coeff(output_term)
        except AttributeError:
            coeff_R = 0
        coeff = coeff_L - coeff_R
        A_row.append(coeff)
        b_term += coeff * output_term
    A.append(A_row)
    b.append(b_term)
A = sp.Matrix(A)
A.simplify()
A = A
b = sp.Matrix([b]).T
b.simplify()
b = b
x = sp.Matrix([outputs]).T
x.simplify()
x = x

In [ ]:
x

In [ ]:
L,U,perm = A.LUdecomposition()

In [ ]:
U

In [ ]:
P = sp.eye(A.rows).permuteFwd(perm)

In [ ]:
y = sp.MatrixSymbol("y", L.shape[0], 1).as_explicit()
y_00 = (P*b)[0]
y_out = y.subs(y[0,0], y_00)
for i in range(1, L.shape[0]):
    y_i0 = y[i,0] - (L*y_out - P*b)[i]
    y_out = y_out.subs(y[i,0], y_i0)
y_out = y_out.simplify()

In [ ]:
assert (L*y_out - P*b).simplify() == sp.zeros(L.shape[0], 1)

In [ ]:
F_CT_idx = list(x).index(F_CT)

In [ ]:
y_out

# LU scratch that works

In [ ]:
y = sp.MatrixSymbol("y", L.shape[0], 1).as_explicit()


$Ly = pB$, so $Ly - Pb=0$ 

In [ ]:
y = sp.MatrixSymbol("y", L.shape[0], 1).as_explicit()
y_00 = (P*b)[0]
y_sub0 = y.subs(y[0,0], y_00)

$L$ has unit diagonl

In [ ]:
y_10 = y[1,0] - (L*y_sub0 - P*b)[1]
y_sub01 = y_sub0.subs(y[1,0], y_10)

# Dump

In [ ]:
A_aug = A.row_join(b)

In [ ]:
L_orig = L.copy()

In [ ]:
L[1,:] - L[1,0]*L[0,:]

In [ ]:
y = sp.MatrixSymbol("y", L.shape[0], 1).as_explicit()
y

In [ ]:
L*y - P*b

In [ ]:
L

In [ ]:
U@x

In [ ]:
L,U,perm = A.LUdecomposition()

In [ ]:
U@x

In [ ]:
L.shape

In [ ]:
P = sp.eye(A.rows).permuteFwd(perm)

In [ ]:
U.shape

In [ ]:
U[22,:]

In [ ]:
P.inv()

In [ ]:
A_aug_sparse = sp.SparseMatrix(A_aug)

In [ ]:
L

In [ ]:
A_aug_sparse[:,21]

In [ ]:
A.rref()

In [ ]:
M[0,0]

In [ ]:
import time

In [ ]:
# print(time.time())
# A_aug.subs([(m_L, 1), (M[0,0], 5), (r, 2), (d_N, 22)])
# print(time.time())


In [ ]:

results = A_aug.rref()[0]
A_aug = A_aug
A_prime = results[:, :-1]
A_prime = A_prime
b_prime = results[:, -1]
A_prime = A_prime
b_prime = b_prime
x = x

F_CN_idx = list(x).index(F_CN)
F_CN_exp = b_prime[F_CN_idx] - (A_prime@x)[F_CN_idx].coeff(a_LN)*a_LNd
F_NL_idx = list(x).index(F_NL)
F_NL_exp = b_prime[F_NL_idx] - (A_prime@x)[F_NL_idx].coeff(a_LN)*a_LNd
N_a_LN_exp = (A_prime@x)[F_CN_idx,0].coeff(a_LN).expand()
alpha_mu_exp = N_a_LN_exp.coeff(mu)
alpha_exp = (N_a_LN_exp - N_a_LN_exp.coeff(mu)*mu).simplify()
N_rhs_exp = (b_prime)[F_CN_idx,0].expand()
gamma_mu_exp = N_rhs_exp.coeff(mu)
gamma_exp = (N_rhs_exp - N_rhs_exp.coeff(mu)*mu).simplify()

F_CT_idx = list(x).index(F_CT)
T_a_LN_exp = (A_prime@x)[F_CT_idx,0].coeff(a_LN).expand()
f_mu_exp = T_a_LN_exp.coeff(mu)
f_exp = (T_a_LN_exp - T_a_LN_exp.coeff(mu)*mu).simplify()
T_rhs_exp = (b_prime)[F_CT_idx,0].expand()
g_mu_exp = T_rhs_exp.coeff(mu)
g_exp = (T_rhs_exp - T_rhs_exp.coeff(mu)*mu).simplify()

tau_M_idx = list(x).index(tau_M)
tau_M_exp = b_prime[F_CT_idx] - (A_prime@x)[F_CT_idx].coeff(a_LN)*a_LNd

get_F_CN = lambdify([alg_inputs], F_CN_exp)
get_F_NL = lambdify([alg_inputs], F_NL_exp)
get_tau_M = lambdify([alg_inputs], tau_M_exp)
get_alpha = lambdify([alg_inputs], alpha_exp)
get_alpha_mu = lambdify([alg_inputs], alpha_mu_exp)
get_gamma = lambdify([alg_inputs], gamma_exp)
get_gamma_mu = lambdify([alg_inputs], gamma_mu_exp)
get_f = lambdify([alg_inputs], f_exp)
get_f_mu = lambdify([alg_inputs], f_mu_exp)
get_g = lambdify([alg_inputs], g_exp)
get_g_mu = lambdify([alg_inputs], g_mu_exp)

tau_M_idx = list(x).index(tau_M)
tau_M_exp = b_prime[tau_M_idx]

In [ ]:
import numpy as np

In [ ]:
sp.Matrix(np.random.rand(23, 26)).rref()

In [ ]:
np.linalg.rref